In [ ]:
# 사용자로부터 입력을 받아서 서버로 전송하는 태그들
'''
    <input name='xxx'
    <select name='xxx'
    <radio name='xxx'
    <checkbox name='xxx'
    <textarea name='xxx'

    xxx가 key가 되고 사용자가 입력(선택, 체크)한값 value가 된다

    selenium에서 값을 세팅하는 방법 => send_keys('값')

    # 네이버 예시
    driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/')
    # 아이디 입력창 획득
    id_input = driver.find_element(By.ID, 'id')
    id_input.send_keys('a-academy')
    pw_input = driver.find_element(By.ID, 'pw')
    pw_input.send_keys('12341234')
    # 로그인
    driver.find_element(By.ID,"log.login").click()
'''

# 기초 작업 

In [4]:
# 타겟사이트 
# https://www.opinet.co.kr/searRgSelect.do

# 전국 주요소 가격 정보(일단위) 수집 : 24:00 수집 목표 (1일 1회)-> 변동시간 확인후 주기 설정
# 시도 > 시군구 > 엑셀다운로드 -> *.xlsx  다운(윈도우 다운로드폴더) -> 다 모아서 df로구성(pandas) -> 엑셀삭제(os 모듈) -> 디비 삽입

import pandas as pd
import os
import sys
from selenium import webdriver as wd
import time
import random # 엑셀을 다운받는 행위에 대한 시간 간격을 변조
import selenium
from selenium.webdriver.common.by import By

selenium.__version__

'4.11.2'

In [2]:
# 브라우저 가동
driver = wd.Chrome()

In [20]:
# 초기 페이지 접속
target_site = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get( target_site )
time.sleep(2) # 사이트가 늦게 뜨면 대기, 디도그 방어코드 (cloudflare => 10초)

# STEP 1 

- 지역 정보 획득 - select 태그

In [29]:
# select#SIDO_NM0 > option => 시도 존재하는 모든 데이터 17(시도) + 1(프럼프트)
sidos = driver.find_elements(By.CSS_SELECTOR, 'select#SIDO_NM0 > option')
len(sidos) # 18

18

In [30]:
for sido in sidos[1:]: # 프럼프트 제외한 나머지 실제 시도(17개)
    value = sido.get_attribute('value').strip()
    print( value, end=', ' )
    #break    

서울특별시, 부산광역시, 대구광역시, 인천광역시, 광주광역시, 대전광역시, 울산광역시, 세종특별자치시, 경기도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도, 제주특별자치도, 강원특별자치도, 

In [31]:
sido_values = [ sido.get_attribute('value').strip() for sido in sidos[1:] ]
print( sido_values )

['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도', '강원특별자치도']


In [23]:
# 사용자가 경기를 클릭햇다 -> select 태그를 특정해서, 부산값을 넣고, send_keys()
SIDO_NM0_SEL = driver.find_element(By.ID,'SIDO_NM0')
# 값입력 -> 샘플로 부산(1)
SIDO_NM0_SEL.send_keys( sido_values[2] )


In [46]:
# 최종코드 
# step 1
target_site = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get( target_site )
time.sleep(2) # 사이트가 늦게 뜨면 대기, 디도그 방어코드 (cloudflare => 10초)

# step 2
sidos = driver.find_elements(By.CSS_SELECTOR, 'select#SIDO_NM0 > option')
sido_values = [ sido.get_attribute('value').strip() for sido in sidos[1:] ]

# step 3
for value in sido_values[:1]:#sidos[1:]:
    # value : 서울특별시 -> 부산광역시 -> ...
    # 지역변경 
    # 하위 시군구에서 화면이 갱신되기 때문에 매번 새로 구한다
    SIDO_NM0_SEL = driver.find_element(By.ID,'SIDO_NM0')
    SIDO_NM0_SEL.send_keys( value )
    # 1초 대기
    time.sleep(1)
    
    # 시군구 값 획득 -> 시군구 선택(for) -> 잠시대기 -> 엑셀저장 클릭
    # 1. 시군구 값 획득
    sigungus = driver.find_elements(By.CSS_SELECTOR, '#SIGUNGU_NM0 > option')
    sigungus = [ opt.get_attribute('value') for opt in sigungus[1:] ]
    #print( sigungus )

    for sigungu in sigungus:#[:3]: # ['강남구', '강동구', ...]
        # 2. 시군구 요소 획득 -> 시군구가 선택되면 메모리가 변경됨(화면갱신)
        tmp = driver.find_element(By.CSS_SELECTOR, '#SIGUNGU_NM0')

        # 3. 시군구 선택(햐나씩) -> form 전송 발생 : 화면 껌벅 : X자 발견 -> 화면갱신
        #    이 결과로 새로운 지역의 주유소 정보가 내려온다
        tmp.send_keys( sigungu )

        # 4. 잠시 대기 (화면껌벅, 새로 화면이 모드 렌더링이 된다)
        time.sleep(1*5)

        # 5. 엑셀 저장 버튼 클릭
        driver.find_element(By.ID, 'glopopd_excel').click()
